# 5 - Associazione di prodotti consigliati

Intersecando l'insieme degli antecedents delle regole di associazione e l'insieme degli items contenuti nell'ultimo acquisto di ognuno dei clienti target, ottengo l'insieme dei prodotti da consigliare per il prossimo acquisto (il consequent della regola). 

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
rules = pd.read_csv('rules.csv')
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,frozenset({'20724'}),frozenset({'22356'}),0.045999,0.034789,0.023966,0.521008,14.976097,0.022366,2.015089
1,frozenset({'22356'}),frozenset({'20724'}),0.034789,0.045999,0.023966,0.688889,14.976097,0.022366,3.066431
2,frozenset({'20725'}),frozenset({'20726'}),0.071898,0.043293,0.024739,0.344086,7.947773,0.021626,1.458585
3,frozenset({'20726'}),frozenset({'20725'}),0.043293,0.071898,0.024739,0.571429,7.947773,0.021626,2.165571
4,frozenset({'20725'}),frozenset({'20727'}),0.071898,0.058369,0.028218,0.392473,6.724026,0.024021,1.549942


In [4]:
buyed_items_per_cust = pd.read_csv('buyed_items_per_customer.csv')

In [5]:
buyed_items_per_cust.head()

,CustomerID,InvoiceDate,StockCode
0,12415.0,2011-01-06 11:12:00,"{'22077', '21915', '21238', '22522', '21891', '22620', '22378', '21914', '22524', '20677', '22492'}"
1,12415.0,2011-02-15 09:52:00,"{'22617', '22716', '21086', '22907', '21058', '23231', '22654', '20973', '21977', '22630', '22613', '21094', '23230', '22417', '21212'}"
2,12415.0,2011-03-03 10:59:00,"{'21238', '22046', '22028', '22654', '22708', '21094', '21984', '21981', '22035', '21914', '22615', '21508', '22961', '22617', '22541', '21915', '21086', '22620', '20677', '22027', '22045', '21509', '22024', '22630', '22993'}"
3,12415.0,2011-05-17 15:42:00,"{'21090', '22918', '22919', '22992', '21899', '21791', '21892', '23253', '21900', '22917', '22928', '23298', '22991', '22549', '22979', '21877', '22916', '21902', '21084', '22921', '22925', '85132A', '22920', '23173'}"
4,12415.0,2011-05-20 14:13:00,{'22712'}


In [6]:
buyed_items_per_cust['InvoiceDate'] = pd.to_datetime(
        buyed_items_per_cust['InvoiceDate'])

In [7]:
buyed_items_per_cust = buyed_items_per_cust.loc[
    buyed_items_per_cust.groupby('CustomerID')['InvoiceDate'].idxmax()]

In [8]:
buyed_items_per_cust['predicted'] = ''
buyed_items_per_cust['confidence'] = ''
buyed_items_per_cust['lift'] = ''

### Costruisco il dataset contenente le associazioni tra clienti e prodotti consigliati, riportando i valori di lift e confidenza delle regole che hanno generato tali risultati.

In [9]:
for i,row in buyed_items_per_cust.iterrows():
    predicted = []
    confidences = []
    lifts = []
    for i2, row2 in rules.iterrows():
        #print(set(row['StockCode'][2:-2].split("', '")))
        #print(set(row2['antecedents'][12:-3].split(' ')))
        subset = set((row['StockCode'][2:-2].split("', '")))
                .intersection(set(row2['antecedents'][12:-3].split(' ')))
        if len(subset) > 0:
            #print(subset)
            predicted.append(row2['consequents'][11:-2])
            confidences.append(row2['confidence'])
            lifts.append(row2['lift'])
    buyed_items_per_cust.loc[buyed_items_per_cust['CustomerID'] == 
                             row['CustomerID'], ['predicted']] =
                                ' / '.join(predicted)[1:-1]
    buyed_items_per_cust.loc[buyed_items_per_cust['CustomerID'] ==
                             row['CustomerID'], ['confidence']]  = 
                                ' / '.join(str(confidences).split(', '))
    buyed_items_per_cust.loc[buyed_items_per_cust['CustomerID'] ==
                             row['CustomerID'], ['lift']]  = 
                                ' / '.join(str(lifts).split(', '))

In [10]:
predictions_df = buyed_items_per_cust[buyed_items_per_cust['predicted'] !=
        ''][['CustomerID', 'StockCode', 'predicted', 'confidence', 'lift']]

In [27]:
predictions_df[['CustomerID','predicted', 'confidence', 'lift']]

,CustomerID,predicted,confidence,lift
10,12415.0,22629,[0.7524752475247525],[15.208230198019804]
23,12590.0,22356' / '22629,[0.5210084033613446 / 0.7524752475247525],[14.976097105508872 / 15.208230198019804]
497,14607.0,22356' / '20724,[0.5210084033613446 / 0.688888888888889],[14.976097105508872 / 14.976097105508872]
673,15061.0,22629,[0.7524752475247525],[15.208230198019804]
1059,16745.0,22356' / '20724,[0.5210084033613446 / 0.688888888888889],[14.976097105508872 / 14.976097105508872]
1203,17677.0,22356,[0.5210084033613446],[14.976097105508872]


In [12]:
df = pd.read_csv('cleaned_online_retail.csv')

In [23]:
#set(predictions_df['predicted'])
single_elems = []
for elem in set(predictions_df['predicted']):
    single_elems.append(elem.split("' / '").pop())

In [24]:
single_elems

['22356', '22629', '20724', '22629']

In [26]:
for el in set(single_elems):
    print(df[df['StockCode']==el].head(1)[['StockCode','Description']])
    print()

     StockCode                  Description
3126     20724  RED RETROSPOT CHARLOTTE BAG

     StockCode                  Description
3140     22356  CHARLOTTE BAG PINK POLKADOT

   StockCode          Description
35     22629  SPACEBOY LUNCH BOX 



# Risultato

Dall'intersezione tra l'insieme degli items contenuti nell'ultimo carrello acquistato e gli antecedents contenuti nelle regole di associazione individuate, abbiamo scoperto, per 6 clienti, la presenza di un prodotto da consigliare per il prossimo acquisto di stock.
I tre prodotti individuati possono quindi essere inviati come campioni ai rispettivi customers, proponendo una offerta per l'acquisto di almeno un certo numero di pezzi, oppure una offerta progressiva in base alla quantità acquistata.

    CLIENTE                  PRODOTTI CONSIGLIATI

    - 12415          SPACEBOY LUNCH BOX
    - 12590          SPACEBOY LUNCH BOX, CHARLOTTE BAG PINK POLKADOT
    - 14607          CHARLOTTE BAG PINK POLKADOT, RED RETROSPOT CHARLOTTE BAG
    - 15061          SPACEBOY LUNCH BOX
    - 16745	      CHARLOTTE BAG PINK POLKADOT, RED RETROSPOT CHARLOTTE BAG
    - 17677          CHARLOTTE BAG PINK POLKADOT